TODO: implemente dashed lines or just corners

## Imports

In [1]:
# numpy for array manipulations
import numpy as np
# image manipulating and saving
from PIL import Image
# more coordinate support
import globalmaptiles as gmaptiles
# for working with metadata in csv
import pandas as pd

map_coords = gmaptiles.GlobalMercator(tileSize=512)

## Global variables

In [2]:
# style id, STYLE_NAME is used in the name of output files 
STYLE_NAME = "nord-light"
#STYLE_NAME = "nord-dark"
#STYLE_NAME = "nord-darker"

# descriptive name for region of interest, used in filename for outputfiles
REGION_NAME = "Zurich-zoom"
FOLDER = "Erasmuskaart/"
# zoomlevel to be used, for more info, see: https://wiki.openstreetmap.org/wiki/Zoom_levels
ZOOM = 14
# min and max longitude and latitude of the region of interest

# size in pixels of 1 tile. Default is 256 unless you did "louche aanpassing" in the sourcecode of cartopy
TILESIZE = 512

# if true, a rectangle is added to the output image displaying the actual region of interest
DISPLAY_REGION_OF_INTEREST = True   # yet to be implemented :)
# define aditional regions to be displayed on the map
LON = (5.95, 10.5)
LAT = (45.81, 47.81)
#REGIONS = [[*LON, *LAT], [8.460907, 8.712447, 47.263396, 47.406200], [8.477787, 8.603557, 47.334798, 47.406200]]
# RGB values of colors to use for boxes
#COLORS = [[172,172,172], [191,97,106], [191,97,106]]
#REGIONS = [[8.460907, 8.712447, 47.263396, 47.406200]]
REGIONS = [[8.477787, 8.603557, 47.334798, 47.406200]]
COLORS = [[191,97,106]]
LINEWIDTH = 4

# you shouldn't need to change these ;)
EARTH_CIRCUMFERENCE = 40_075_016.686   # meter

## Local functions

In [3]:
def LatLonToMapPixel(lat, lon, zoom, map_pixel_origin):
    # convert lon-lat to meters
    xm, ym = map_coords.LatLonToMeters(lat, lon)
    # convert meters to pixel
    px, py = map_coords.MetersToPixels(xm, ym, zoom)
    mpx = px-map_pixel_origin[0]
    mpy = map_pixel_origin[1]-py   # reverse because lat coordinates are in reverse wrt pseudo mercator
                                   # might have to be fixed for southern hemisphere, idk
    #mpy = py-map_pixel_origin[1]
    
    return int(np.floor(mpx)), int(np.floor(mpy))

## Fetch mapdata

Fetch data from csv

In [4]:
FILE_NAME = "%s_%s_%s"%(REGION_NAME,STYLE_NAME,ZOOM)
SEP = ";"
df = pd.read_csv("maps_metadata.csv", sep=SEP)
df = df.set_index("MapName")
MAP_DATA = df.loc[FILE_NAME]
MAP_DATA

Region                          Zurich-zoom
StyleName                        nord-light
StyleID           ckylx55c21qne14nvbv5zkksv
IsPrivateStyle                         True
user                             cubedsheep
LatMin                            47.334798
LatMax                              47.4062
LonMin                             8.477787
LonMax                             8.603557
Zoom                                     14
TileSize                                512
PixWidth                               3584
PixHeight                              3072
PixScaleTop                        3.233269
PixScaleBottom                     3.237649
MeterXMin                     941704.188473
MeterXMax                     958826.082809
MeterYMin                    5995109.002463
MeterYMax                    6009784.911894
MapLatMin                         47.323931
MapLatMax                          47.41322
MapLonMin                          8.459473
MapLonMax                       

In [5]:
# extract required data
LAT = (MAP_DATA["LatMin"], MAP_DATA["LatMax"])
LON = (MAP_DATA["LonMin"], MAP_DATA["LonMax"])
IMG_SHAPE = (MAP_DATA["PixHeight"], MAP_DATA["PixWidth"])
PIX_ORIGIN = (MAP_DATA["MapPixXMin"], MAP_DATA["MapPixYMax"])

MAP_SCALE = MAP_DATA["PixScaleTop"]

## Load map image and add regions

In [6]:
# load image
image = Image.open(FOLDER+FILE_NAME+".png")
# also get representation as numpy array for easier pixel manipulations
img_arr = np.array(image)

Convert coordinates of regions to pixel indicies on map

In [7]:
pixel_regions = []
for region in REGIONS:
    pixel1 = LatLonToMapPixel(region[3], region[1], ZOOM, PIX_ORIGIN)
    pixel2 = LatLonToMapPixel(region[2], region[0], ZOOM, PIX_ORIGIN)
    pixel_regions.append([pixel2[0],pixel1[0],pixel1[1],pixel2[1]])

Add rectangles for regions

In [8]:
for i in range(len(REGIONS)):
    region = pixel_regions[i]
    color = COLORS[i]
    rows = (region[2],region[3]+1)
    cols = (region[0],region[1]+1)
    # draw vertical lines, draw outside the box
    img_arr[(rows[0]-LINEWIDTH):rows[0],(cols[0]-LINEWIDTH):(cols[1]+LINEWIDTH)] = color
    img_arr[rows[1]:(rows[1]+LINEWIDTH),(cols[0]-LINEWIDTH):(cols[1]+LINEWIDTH)] = color
    img_arr[(rows[0]-LINEWIDTH):(rows[1]+LINEWIDTH),(cols[0]-LINEWIDTH):cols[0]] = color
    img_arr[(rows[0]-LINEWIDTH):(rows[1]+LINEWIDTH),cols[1]:(cols[1]+LINEWIDTH)] = color

convert array back to image and save it

In [9]:
region_image = Image.fromarray(img_arr)
region_image.save("%s-regions.png"%FILE_NAME, "png")